In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import re
import pickle
from timeit import default_timer as timer
import sklearn
from sklearn.preprocessing import scale
from sklearn.manifold import Isomap, TSNE
from sklearn.metrics import silhouette_samples, silhouette_score
import scipy
import scipy.cluster.hierarchy
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import umap
import seaborn as sns
import matplotlib.cm as cm
import random

from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import mannwhitneyu

# from pysankey import sankey

from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN, SpectralClustering, Birch
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_selection import mutual_info_classif
from sklearn import metrics
import igraph
from math import ceil
from timeit import default_timer as timer
import subprocess
from pathlib import Path
from numpy import linalg as LA

import neuro_morpho_toolbox as nmt
from neuro_morpho_toolbox.ml_utilities import *
from neuro_morpho_toolbox.plot import qualitative_scatter, cell_in_map

import SimpleITK as sitk

from ipywidgets import interactive
import ipywidgets as widgets

import plotly.plotly as py
import plotly.offline as po
import plotly.graph_objs as go

po.init_notebook_mode(connected=True)

import colorlover as cl
from IPython.display import HTML

bs_child_to_parent = {}

for pid in nmt.bs.selected_regions:
    pname = nmt.bs.level.loc[pid, "Abbrevation"]
    cname = [nmt.bs.level.loc[i, "Abbrevation"] for i in nmt.bs.get_all_child_id(pid)]
    for i in cname:
        bs_child_to_parent[i] = pname
bs_child_to_parent["unknown"] = "unknown"

/Users/pengxie/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


/Users/pengxie/Documents/Research/python/neuro_morhpo_toolbox/neuro_morpho_toolbox/
Loading CCF Atlas data...
Loading time: 0.85
Loading CCF brain structure data...
Loading time: 0.01


/Users/pengxie/.local/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


## Load data

In [2]:
[metadata] = pickle.load(open("metadata_cluster.pickle", 'rb'))
# metadata = metadata[~metadata.Celltype.isin(["CPU", "Others"])]
# metadata = metadata[~metadata.cluster.isin(["NA"])]
all_cells = metadata.index.tolist()
metadata['Brain_id'] = ['Brain_'+i.split("_")[0] for i in all_cells]
metadata['SWC'] = ['../Data/swc/mapped/Data_for_analysis/swc/Whole/'+i+'.swc' for i in all_cells]
# metadata['Celltype'] = 'UnAssigned'
# metadata['Subtype'] = 'UnAssigned'

metadata.Celltype.value_counts()

VPM             34
LGd             34
MG              21
VPL             19
LP               6
TH_Uncertain     5
MD               2
VAL              2
AM               2
VM               1
IMD              1
PO               1
PCN              1
Name: Celltype, dtype: int64

In [3]:
# celltype_color_dict = nmt.get_group_colors(metadata, "Celltype")
[celltype_color_dict] = pickle.load(open("colors.pickle", "rb"))
cluster_color_dict = nmt.get_group_colors(metadata, 'cluster')
brain_color_dict = nmt.get_group_colors(metadata, 'Brain_id')

In [4]:
[soma] = pickle.load(open("features_soma.pickle", "rb"))
[projection] = pickle.load(open("features_projection.pickle", "rb"))
[lm_axon] = pickle.load(open("features_lm_axon.pickle", "rb"))
[lm_dendrite] = pickle.load(open("features_lm_dendrite.pickle", "rb"))

In [5]:
projection_df = projection.scaled_data.copy() - np.log(100)
projection_df = projection_df.loc[all_cells]
projection_df = projection_df.round(2)
projection_df.drop(columns=['ipsi_fiber tracts', 'contra_fiber tracts'], inplace=True)
use_features = projection_df.columns[np.sum(projection_df>0)>=3]
projection_df = projection_df[use_features]
print("Projection features: %d" % (projection_df.shape[1]))

Projection features: 62


In [6]:
soma_df = soma.raw_data.copy().loc[all_cells]

In [7]:
lm_axon_df = lm_axon.raw_data.copy().loc[all_cells]
use_cols = ['A_Overall_Width', 
            'A_Overall_Height', 
            'A_Overall_Depth', 
            'A_Total_Length', 
            'A_Max_Euclidean_Distance', 
            'A_Max_Path_Distance', 
            'A_Number_of_Branches', 
#             'Max Branch Order',
#             'Average Contraction', 
#             'Average Fragmentation',
#             'Average Bifurcation Angle Local', 
#             'Average Bifurcation Angle Remote', 
#             'Hausdorff Dimension',
            'AL_Total_Length',
            'AL_Number_of_Branches'
           ]
lm_axon_df = lm_axon_df[use_cols]
lm_axon_df_scale = pd.DataFrame(scale(lm_axon_df), 
                                index=lm_axon_df.index, 
                                columns=lm_axon_df.columns)


In [8]:
lm_dendrite_df = lm_dendrite.raw_data.copy().loc[all_cells]
use_cols = [
    'D_Number_of_Stems', 
    'D_Overall_Width', 
    'D_Overall_Height', 
    'D_Overall_Depth', 
    'D_Total_Length',
    'D_Max_Euclidean_Distance', 
    'D_Max_Path_Distance', 
    'D_Number_of_Branches', 
#     'D_Max_Branch_Order', # Add back after dendrite correction
]
lm_dendrite_df = lm_dendrite_df[use_cols]
lm_dendrite_df["D_Depth_Width-Ratio"] = lm_dendrite_df["D_Overall_Depth"] / lm_dendrite_df["D_Overall_Width"]
lm_dendrite_df_scale = pd.DataFrame(scale(lm_dendrite_df), 
                                    index=lm_dendrite_df.index, 
                                    columns=lm_dendrite_df.columns
                                   )


In [10]:
projection_df.loc['18457_00013'].sort_values(ascending=False).head(10)

ipsi_SSp-n      6.29
ipsi_SSp-bfd    5.87
ipsi_CP         3.94
ipsi_SSs        2.67
ipsi_SSp-un     1.52
ipsi_LGv        0.00
ipsi_VISl       0.00
ipsi_VISp       0.00
ipsi_VISpl      0.00
ipsi_VISpm      0.00
Name: 18457_00013, dtype: float64